<a href="https://colab.research.google.com/github/kartozia/test/blob/master/edit_distance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

↑ Чтобы работать с этим ноутбуком, нажмите *Open in playground* ↑

Google Colaboratory позволяет запускать код прямо в браузере, а файлы брать с гугл-диска. Но сначала диск нужно подключить: запустите кусочек кода ниже, пройдите по ссылке, которая под ним появится, дайте доступ приложению Google Drive File Stream, скопируйте ключ, вставьте его в поле под ссылкой и нажмите Enter.

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


Устанавливаем fuzzywuzzy. Библиотека быстрее работает на основе другого пакета - python-Levenshtein. Следующий кусочек кода устанавливает их обе.

In [0]:
!pip install fuzzywuzzy[speedup]

In [0]:
import pandas as pd
import re
from fuzzywuzzy import fuzz
import string

In [0]:
df = pd.read_csv('gdrive/My Drive/edit_distance/greedy.csv') #  Загружаем .csv-файл с данными. Предполагается, что он лежит на гугл-диске в папке edit_distance

In [0]:
df.head()

,date,text,place
0,3/2/2019 1:11:19,"солёный огурец, на полу валяется, никто её не ест","Казань 1984-1991, Ульяновск 1991-1998"
1,3/2/2019 1:19:18,"- солёный огурец! На полу валялся, никто его н...","г. Кемерово, Кемеровская обл."
2,3/2/2019 1:19:51,- турецкий барабан.,"г. Кемерово, Кемеровская обл."
3,3/2/2019 1:41:40,"Соленый огурец, на полу валяется, никто его не...",Москва
4,3/2/2019 1:41:56,"Жадина говядина соленый огурец,на полу валяетс...","Оренбургская область,Бугуруслан"


Пишем функцию для разбиения фрагментов текста на слова с использованием магии регулярных выражений

In [0]:
def tokenize(text):
  return re.findall(r'[^\w\s]+|\w+', text)

Она принимает на вход текст и разбивает его на слова и знаки препинания:

In [0]:
tokenize("солёный огурец, на полу валяется, никто её не ест")

['солёный',
 'огурец',
 ',',
 'на',
 'полу',
 'валяется',
 ',',
 'никто',
 'её',
 'не',
 'ест']

Теперь можно искать огурцы, задействуя расстояние Левенштейна: сравним каждое слово в строке, разбитой на токены, с искомым. 

fuzz.ratio выдает число, которое характеризует степень схожести слов.

В функции ниже задано условие: выбрать только те слова, для которых степень схожести с исходным не меньше 75.

In [0]:
def find_variation(tokenized_string, standard):
  for word in tokenized_string:
    if fuzz.ratio(standard, word) > 75: # Можно поменять это значение и посмотреть, что будет.
      return word

In [0]:
find_variation(tokenize("солёный огурец, на полу валяется, никто её не ест"), "огурец")

'огурец'

Теперь напишем функцию, которая пройдется по всей таблице и поймает вариации на тему искомого слова.

In [0]:
def find_all_variations(standard, df):
  variations = []
  for index, row in df.iterrows():
    row[1] = row[1].lower().replace('ё', 'е') # делаем все буквы в строке строчными и заменяем "ё" на "е"
    tokenized = tokenize(row[1])
    variation = find_variation(tokenized, standard)
    if variation:
      variations.append(variation)
  return set(variations)

Посмотрим, как работает все вместе:

In [0]:
standard = 'огурец'
variations = set(find_all_variations(standard, df))
print(variations)

{'огуоец', 'огурцом', 'огурец'}


Вариантов написания слова "огурец" не так много, зато какое разнообразие "соленых"!

In [0]:
standard = 'соленый'
variations = set(find_all_variations(standard, df))
print(variations)

{'соленым', 'солоный', 'cоленый', 'соленый', 'соленный', 'солный', 'слоеный'}


В этом списке смущают две вещи: 

во-первых, 'слоеный', но в данном случае будем считать его тоже опечаткой,

во-вторых, похожие как две капли воды 'соленый' и 'cоленый'. Видимо, чем-то они все же различаются, посмотрим не затесалось ли там латиницы.

In [0]:
for word in variations:
  for letter in word:
    if letter in string.ascii_letters:
      print("Нашлась латинская буква \"{}\" в слове \"{}\"".format(letter, word))

Нашлась латинская буква "c" в слове "cоленый"


Теперь можно приводить все это разнообразие к единому написанию.

In [0]:
variations_to_replace = {
    'огуоец': 'огурец',
    'соленный': 'соленый',
    'солный': 'соленый',
    'слоеный': 'соленый',
    'солоный': 'соленый',
    'cоленый': 'соленый' # заменяем слово с латиницей на слово с кириллицей, хотя выглядят они одинаково
}

df["text"].replace(variations_to_replace, regex=True, inplace=True)

Проверим, найдется ли в табличке слово "солоный". После замены на "соленый" его там быть не должно и следующий код должен выдать пустую табличку.

In [0]:
df[df['text'].str.contains('солоный')]

,date,text,place


В целом же табличка, как можно убедиться, не пострадала:

In [0]:
df.head()

,date,text,place
0,3/2/2019 1:11:19,"соленый огурец, на полу валяется, никто ее не ест","Казань 1984-1991, Ульяновск 1991-1998"
1,3/2/2019 1:19:18,"- соленый огурец! на полу валялся, никто его н...","г. Кемерово, Кемеровская обл."
2,3/2/2019 1:19:51,- турецкий барабан.,"г. Кемерово, Кемеровская обл."
3,3/2/2019 1:41:40,"соленый огурец, на полу валяется, никто его не...",Москва
4,3/2/2019 1:41:56,"жадина говядина соленый огурец,на полу валяетс...","Оренбургская область,Бугуруслан"
